# Evaluate 
$ f(x, y) = 2x^2 - x^2y^2 + 3 $

$ g_0 = x ⋅ x $ 

$ g_1 = x ⋅ x $ 

$ g_2 = y ⋅ y $

$ g_3 = g_0 ⋅ 2 $

$ g_4 = g_1 ⋅ g_2 $

$ g_5 = g_2 - g_4 $

$ g_6 = g_4 + 3 $

---

$ c_0 = a_0 ⋅ b_0 $

$ c_1 = a_1 ⋅ b_1 $

$ c_2 = a_2 ⋅ b_2 $

$ c_3 = a_3 ⋅ 2 $

$ c_4 = a_4 ⋅ b_4 $

$ c_5 = a_5 - b_5 $

$ c_6 = a_6 + 3 $

---

$ x = a_0 = b_0 = a_1 = b_1 $

$ y = a_2 = b_2 $

$ g_0 = a_3 = c_0 $

$ g_1 = a_4 = c_1 $

$ g_2 = b_4 = c_2 $

$ g_3 = a_5 = c_3 $

$ g_4 = b_5 = c_4 $

$ g_5 = a_6 = c_5 $

$ g_6 = c_6 $

### Gate constrain equation

$ g_i = q_{Li}⋅a_i+q_{Ri}⋅b_i+q_{Mi}⋅(a_i⋅b_i)+q_{Ci}+q_{Oi}⋅c_i=0 $

$ g_0 = 0⋅a_0 + 0⋅b_0 + 1⋅(a_0⋅b_0) + 0 + (-1)⋅c_0 = 0 $

$ g_1 = 0⋅a_1 + 0⋅b_1 + 1⋅(a_1⋅b_1) + 0 + (-1)⋅c_1 = 0 $

$ g_2 = 0⋅a_2 + 0⋅b_2 + 1⋅(a_2⋅b_2) + 0 + (-1)⋅c_2 = 0 $

$ g_3 = 2⋅a_3 + 0⋅b_3 + 0⋅(a_3⋅b_3) + 0 + (-1)⋅c_3 = 0 $

$ g_4 = 0⋅a_4 + 0⋅b_4 + 1⋅(a_4⋅b_4) + 0 + (-1)⋅c_4 = 0 $

$ g_5 = 1⋅a_5 + (-1)⋅b_5 + 0⋅(a_5⋅b_5) + 0 + (-1)⋅c_5 = 0 $

$ g_6 = 1⋅a_6 + 0⋅b_6 + 0⋅(a_6⋅b_6) + 3 + (-1)⋅c_6 = 0 $

**selector vectors**

$ g_i = [q_{Li}, q_{Ri}, q_{Mi}, q_{Ci}, q_{Oi}] $

$ g_0 = [0, 0, 1, 0, -1] $

$ g_1 = [0, 0, 1, 0, -1] $

$ g_2 = [0, 0, 1, 0, -1] $

$ g_3 = [2, 0, 0, 0, -1] $

$ g_4 = [0, 0, 1, 0, -1] $

$ g_5 = [1, -1, 0, 0, -1] $

$ g_6 = [1, 0, 0, 3, -1] $

**let x = 2 y = 3**


| Vectors| a | b | c | qLi | qRi | qMi | qCi | qOi |
| --- | --- | --- | --- |--- | --- | --- | --- | --- |
| gate0 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 |
| gate1 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 |
| gate2 | 3 | 3 | 9 | 0 | 0 | 1 | 0 | -1 |
| gate3 | 4 | 0 | 8 | 2 | 0 | 0 | 0 | -1 |
| gate4 | 4 | 9 | 36 | 0 | 0 | 1 | 0 | -1 |
| gate5 | 8 | 36 | -28 | 1 | -1 | 0 | 0 | -1 |
| gate6 | -28 | 3 | -25 | 1 | 0 | 0 | 3 | -1 |

In [1]:
import galois
import numpy as np

x = 2
y = 3

out = 2*x**2 - x**2*y**2 + 3
print(out)

a = [2, 2, 3, 4, 4, 8, -28]
b = [2, 2, 3, 0, 9, 36, 3]
c = [4, 4, 9, 8, 36, -28, -25]

ql = [0, 0, 0, 2, 0, 1, 1]
qr = [0, 0, 0, 0, 0, -1, 0]
qm = [1, 1, 1, 1, 1, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 3]
qo = [-1, -1, -1, -1, -1, -1, -1]

def to_galois_array(a, field):
    # normalize to positive values
    a = [x % field.order for x in a]
    return GF(a)

def to_vanishing_poly(roots, field):
    # Z^n - 1 = (Z - 1)(Z - w)(Z - w^2)...(Z - w^(n-1))
    return galois.Poly.Degrees([len(roots), 0], coeffs=[1, -1], field=field)

def to_poly(x, v, field):
    assert len(x) == len(v)
    y = to_galois_array(v, field) if type(v) == list else v
    return galois.lagrange_poly(x, y)

def pad_array(a, n):
    return a + [0]*(n - len(a))

n = len(a)
# we have 7 gates, but we need to pad to 2^k
n = 2**int(np.ceil(np.log2(n)))

# we have to pad the arrays to the same length
a = pad_array(a, n)
b = pad_array(b, n)
c = pad_array(c, n)
ql = pad_array(ql, n)
qr = pad_array(qr, n)
qm = pad_array(qm, n)
qc = pad_array(qc, n)
qo = pad_array(qo, n)

p = 241
GF = galois.GF(p)
w = GF.primitive_root_of_unity(n)
tau = GF(12)

roots = GF([w**x for x in range(0, n)])

_A = to_poly(roots, a, GF)
_B = to_poly(roots, b, GF)
_C = to_poly(roots, c, GF)
ZH = to_vanishing_poly(roots, GF)

# a bit awkward, but we need to generate random values in GF for blinding polynomials
# b1, .., b9
b = galois.Poly.Random(degree=10, field=GF).coeffs

bA = galois.Poly(b[:2], field=GF)
bB = galois.Poly(b[2:4], field=GF)
bC = galois.Poly(b[4:6], field=GF)

A = _A + bA*ZH
B = _B + bB*ZH
C = _C + bC*ZH

QL = to_poly(roots, ql, GF)
QR = to_poly(roots, qr, GF)
QM = to_poly(roots, qm, GF)
QC = to_poly(roots, qc, GF)
QO = to_poly(roots, qo, GF)

# gate constraints polynomial
# g(x) = a(x)*ql(x) + b(x)*qr(x) + a(x)*b(x)*qm(x) + qc(x) + c(x)*qo(x)
G = A*QL + B*QR + A*B*QM + QC + C*QO

print("--- Vanishing Polynomial ---")
print("ZH = ", ZH)

print("\n\n--- Blinding Polynomials ---")
print(f"bA = {bA}")
print(f"bB = {bB}")
print(f"bC = {bC}")

print("\n\n--- Wire Polynomials ---")
print(f"A = {_A}")
print(f"B = {_B}")
print(f"C = {_C}")

print("\n\n--- Blinded Wire Polynomials ---")
print(f"A' = {A}")
print(f"B' = {B}")
print(f"C' = {C}")

print("\n\n--- Selector Polynomials ---")
print(f"QL = {QL}")
print(f"QR = {QR}")
print(f"QM = {QM}")
print(f"QC = {QC}")
print(f"QO = {QO}")

print("\n\n--- Gate Constraints Polynomial ---")
print(f"G = {G}")
for i in range(0, len(roots)):
    print(f"gate #{i} G({roots[i]}) = {G(roots[i])} --> {'OK' if G(roots[i]) == 0 else 'FAIL'}")

assert G % ZH == 0

# TODO: calculate [A], [B], [C]

round1 = [A(tau), B(tau), C(tau)]
print("\n\n--- Round 1 ---")
print(f"Round 1 = {round1}")

-25


OMP: Info #270: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


--- Vanishing Polynomial ---
ZH =  x^8 + 240


--- Blinding Polynomials ---
bA = 59x + 236
bB = 126x + 123
bC = 193x + 48


--- Wire Polynomials ---
A = 155x^7 + 227x^6 + 76x^5 + 26x^4 + 192x^3 + 82x^2 + 58x + 150
B = 203x^7 + 28x^6 + 176x^5 + 148x^4 + 217x^3 + 154x^2 + 3x + 37
C = 93x^7 + 22x^6 + 89x^5 + 5x^4 + 78x^3 + 233x^2 + 206x + 1


--- Blinded Wire Polynomials ---
A' = 59x^9 + 236x^8 + 155x^7 + 227x^6 + 76x^5 + 26x^4 + 192x^3 + 82x^2 + 240x + 155
B' = 126x^9 + 123x^8 + 203x^7 + 28x^6 + 176x^5 + 148x^4 + 217x^3 + 154x^2 + 118x + 155
C' = 193x^9 + 48x^8 + 93x^7 + 22x^6 + 89x^5 + 5x^4 + 78x^3 + 233x^2 + 13x + 194


--- Selector Polynomials ---
QL = 187x^7 + 38x^6 + 119x^5 + 60x^4 + 70x^3 + 22x^2 + 106x + 121
QR = 64x^7 + 8x^6 + x^5 + 211x^4 + 177x^3 + 233x^2 + 240x + 30
QM = 57x^7 + 211x^6 + 73x^5 + 211x^4 + 168x^3 + 211x^2 + 184x + 91
QC = 24x^7 + 90x^6 + 217x^5 + 151x^4 + 24x^3 + 90x^2 + 217x + 151
QO = 240x^7 + 8x^6 + 177x^5 + 30x^4 + x^3 + 233x^2 + 64x + 210


--- Gate Constra

### Permutation constraint

In [2]:
import sha3


def numbers_to_hash(numbers: [int], field) -> int:
    """Hash a number."""
    return field(int(sha3.keccak_256(bytes(numbers)).hexdigest(), 16) % field.order)

def print_sigma(sigma, a, b, c, r):
    group_size = len(sigma) // 3
    padding = 6

    print(f"{' w'} | {'value':{padding}} | {'i':{padding}} | {'sigma(i)':{padding}}")

    for i in range(0, group_size):
        print(f"a{i} | {a[i]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(group_size, 2 * group_size):
        print(f"b{i - group_size} | {b[i - group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(2 * group_size, 3 * group_size):
        print(f"c{i - 2 * group_size} | {c[i - 2 * group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

# TODO: calculate beta, gamma from [A], [B], [C]
beta = numbers_to_hash(round1 + [0], GF)
gamma = numbers_to_hash(round1 + [1], GF)

ai = range(0, n)
bi = range(n, 2*n)
ci = range(2*n, 3*n)

sigma = {
    ai[0]: ai[0], ai[1]: ai[1], ai[2]: ai[2], ai[3]: ci[0], ai[4]: ci[1], ai[5]: ci[3], ai[6]: ci[5], ai[7]: ai[7],
    bi[0]: bi[0], bi[1]: bi[1], bi[2]: bi[2], bi[3]: bi[3], bi[4]: ci[2], bi[5]: ci[4], bi[6]: bi[6], bi[7]: bi[7],
    ci[0]: ai[3], ci[1]: ai[4], ci[2]: bi[4], ci[3]: ai[5], ci[4]: bi[5], ci[5]: ai[6], ci[6]: ci[6], ci[7]: ci[7],
}

c1_roots = roots
c2_roots = roots * 2
c3_roots = roots * 4

r = np.concatenate((c1_roots, c2_roots, c3_roots))

check = set()
for w in r:
    if int(w) in check:
        print(f"Duplicate root {w} in {r}")
        assert False
    check.add(int(w))

sigma1 = GF([r[sigma[i]] for i in range(0, n)])
sigma2 = GF([r[sigma[i + n]] for i in range(0, n)])
sigma3 = GF([r[sigma[i + 2 * n]] for i in range(0, n)])

print_sigma(sigma, a, b, c, r)

print("\n\n--- Cosest ---")
print(f"c0 = {c1_roots}")
print(f"c1 = {c2_roots}")
print(f"c2 = {c3_roots}")

print("\n\n--- Sigma ---")
print(f"sigma1 = {sigma1}")
print(f"sigma2 = {sigma2}")
print(f"sigma3 = {sigma3}")

S1 = to_poly(roots, sigma1, GF)
S2 = to_poly(roots, sigma2, GF)
S3 = to_poly(roots, sigma3, GF)

I1 = to_poly(roots, c1_roots, GF)
I2 = to_poly(roots, c2_roots, GF)
I3 = to_poly(roots, c3_roots, GF)

padding = 3
for i in range(0, len(roots)):
    s = f"i = {i:{padding}} --> {roots[i]:{padding}} "
    s += f"   A({roots[i]:{padding}}) = {A(roots[i]):{padding}} "
    s += f"   B({roots[i]:{padding}}) = {B(roots[i]):{padding}} "
    s += f"   C({roots[i]:{padding}}) = {C(roots[i]):{padding}} "
    s += f"   I1({roots[i]:{padding}}) = {I1(roots[i]):{padding}} "
    s += f"   I2({roots[i]:{padding}}) = {I2(roots[i]):{padding}} "
    s += f"   I3({roots[i]:{padding}}) = {I3(roots[i]):{padding}} "
    s += f"   S1({roots[i]:{padding}}) = {S1(roots[i]):{padding}} "
    s += f"   S2({roots[i]:{padding}}) = {S2(roots[i]):{padding}} "
    s += f"   S3({roots[i]:{padding}}) = {S3(roots[i]):{padding}} "
    print(s)


f = (A + I1 * beta + gamma) * (B + I2 * beta + gamma) * (C + I3 * beta + gamma)
g = (A + S1 * beta + gamma) * (B + S2 * beta + gamma) * (C + S3 * beta + gamma)

print("\n\n--- f(x) ---")
print(f"f(x) = {f}")

print("\n\n--- g(x) ---")
print(f"g(x) = {g}")

acc_poly_eval = [GF(1)]
for i in range(0, n):
    acc_poly_eval.append(
        acc_poly_eval[-1] * f(roots[i]) / g(roots[i])
    )

assert acc_poly_eval.pop() == GF(1)
ACC = galois.lagrange_poly(roots, GF(acc_poly_eval))

print("\n\n--- Accumulator Polynomial ---")
print(f"ACC(x) = {ACC}")

# blinding polynomial for accumulator
bZ = galois.Poly(b[6:], field=GF)
Z = bZ * ZH + ACC

print("\n\n--- Z ---")
print(f"Z(x) = {Z}")

for r in roots:
    print(f"Z({r}) = {Z(r)}")

assert Z(roots[0]) == 1
assert Z(roots[-1]) == 1

round2 = [Z(tau)]
print("\n\n--- Round 2 ---")
print(f"Round 2 = {round2}")

 w | value  | i      | sigma(i)
a0 |      2 |      1 |      1
a1 |      2 |     30 |     30
a2 |      3 |    177 |    177
a3 |      4 |      8 |      4
a4 |      4 |    240 |    120
a5 |      8 |    211 |     32
a6 |    -28 |     64 |    121
a7 |      0 |    233 |    233
-- | --     | --     | --    
b0 |     59 |      2 |      2
b1 |    236 |     60 |     60
b2 |    126 |    113 |    113
b3 |    123 |     16 |     16
b4 |    193 |    239 |    226
b5 |     48 |    181 |    237
b6 |    143 |    128 |    128
b7 |    217 |    225 |    225
-- | --     | --     | --    
c0 |      4 |      4 |      8
c1 |      4 |    120 |    240
c2 |      9 |    226 |    239
c3 |      8 |     32 |    211
c4 |     36 |    237 |    181
c5 |    -28 |    121 |     64
c6 |    -25 |     15 |     15
c7 |      0 |    209 |    209


--- Cosest ---
c0 = [  1  30 177   8 240 211  64 233]
c1 = [  2  60 113  16 239 181 128 225]
c2 = [  4 120 226  32 237 121  15 209]


--- Sigma ---
sigma1 = [  1  30 177   4 120  32 121 

In [3]:
# TODO: calculate [Z]
alpha = numbers_to_hash(round1 + round2, GF)

def eval_poly(poly: galois.Poly, domain, shift=GF(1)):
    coeff = poly.coeffs[::-1]
    eval = []
    for j in range(len(domain)):
        eval += [sum([(domain[j] * shift) ** i * coeff[i] for i in range(poly.degree+1)], GF(0))]
    return GF(eval)

def shift_poly(poly: galois.Poly, domain):
    shifted_eval = eval_poly(poly, domain, shift=domain[1])
    return galois.lagrange_poly(domain, shifted_eval)

L1 = galois.lagrange_poly(roots, GF([1] + [GF(0)] * (n - 1)))

# Find Z offset
# roots[1] is a primitive root of unity and a generator of the subgroup
ZSH = shift_poly(Z, roots)
for r in roots:
    print(f"ZSH({r}) = {ZSH(r)}")

T0 = G // ZH
assert G % ZH == 0

T1 = (f * Z - g * ZSH) * alpha
assert T1 % ZH == 0

T2 = (Z - GF(1)) * L1 * alpha**2
assert T2 % ZH == 0

T = (T0 + T1 + T2) // ZH

print("\n\n--- T ---")
print(f"T(x) = {T}")

t_coeffs = T.coeffs[::-1]

Tl = galois.Poly(t_coeffs[:n][::-1], field=GF)
Tm = galois.Poly(t_coeffs[n:2*n][::-1], field=GF)
Th = galois.Poly(t_coeffs[2*n:][::-1], field=GF)

print("\n\n--- T' ---")
print(f"Tl(x) = {Tl}")
print(f"Tm(x) = {Tm}")
print(f"Th(x) = {Th}")

X_n = galois.Poly.Degrees([n, 0], coeffs=[1, 0], field=GF)
X_2n = galois.Poly.Degrees([2*n, 0], coeffs=[1, 0], field=GF)
# make sure that T was split correctly
# T = TL + X^n * TM + X^2n * TH
assert T == (Tl + X_n * Tm + X_2n * Th)

TL = Tl + b[8] * X_n
TM = Tm - b[8] + b[9] * X_n
TH = Th - b[9]

assert T == (TL + X_n * TM + X_2n * TH)

# TODO: calculate [TL], [TM], [TH]

round3 = [TL(tau), TM(tau), TH(tau)]
print("\n\n--- Round 3 ---")
print(f"Round 3 = {round3}")

ZSH(1) = 220
ZSH(30) = 51
ZSH(177) = 111
ZSH(8) = 216
ZSH(240) = 34
ZSH(211) = 146
ZSH(64) = 1
ZSH(233) = 1


--- T ---
T(x) = 154x^31 + 191x^30 + 193x^29 + 174x^28 + 207x^27 + 107x^26 + 67x^25 + 106x^24 + 130x^23 + 90x^22 + 31x^21 + 68x^20 + 76x^19 + 100x^18 + 203x^17 + 154x^16 + 181x^15 + 52x^14 + 195x^13 + 83x^12 + 38x^11 + 14x^10 + 117x^9 + 14x^8 + 108x^7 + 228x^6 + 154x^5 + 157x^4 + 195x^3 + 2x^2 + 231x + 240


--- T' ---
Tl(x) = 108x^7 + 228x^6 + 154x^5 + 157x^4 + 195x^3 + 2x^2 + 231x + 240
Tm(x) = 181x^7 + 52x^6 + 195x^5 + 83x^4 + 38x^3 + 14x^2 + 117x + 14
Th(x) = 154x^15 + 191x^14 + 193x^13 + 174x^12 + 207x^11 + 107x^10 + 67x^9 + 106x^8 + 130x^7 + 90x^6 + 31x^5 + 68x^4 + 76x^3 + 100x^2 + 203x + 154


--- Round 3 ---
Round 3 = [GF(104, order=241), GF(175, order=241), GF(185, order=241)]


In [4]:
# TODO: calculate zeta from [TL], [TM], [TH]
zeta = numbers_to_hash(round1 + round2 + round3, GF)

A_eval = A(zeta)
B_eval = B(zeta)
C_eval = C(zeta)
S1_eval = S1(zeta)
S2_eval = S2(zeta)
ZSH_eval = ZSH(zeta)

print("Openings:")
print(f"A(zeta) = {A_eval}")
print(f"B(zeta) = {B_eval}")
print(f"C(zeta) = {C_eval}")
print(f"S1(zeta) = {S1_eval}")
print(f"S2(zeta) = {S2_eval}")

round4 = [A_eval, B_eval, C_eval, S1_eval, S2_eval, ZSH_eval]
print("\n\n--- Round 4 ---")
print(f"Round 4 = {round4}")

Openings:
A(zeta) = 33
B(zeta) = 88
C(zeta) = 5
S1(zeta) = 237
S2(zeta) = 115


--- Round 4 ---
Round 4 = [GF(33, order=241), GF(88, order=241), GF(5, order=241), GF(237, order=241), GF(115, order=241), GF(3, order=241)]


In [5]:
# TODO compute v from all the values
# opening challenge v
v = numbers_to_hash(round1 + round2 + round3 + round4, GF)


# Compute linearisation polynomial r(X)
R0 = A_eval * B_eval * QM + A_eval * QL + B_eval * QR + C_eval * QO + QC

R1 = alpha * ((A_eval + beta * zeta + gamma)*(B_eval + beta * 2 * zeta + gamma) * (C_eval + beta * 4 * zeta + gamma) * Z - \
    (A_eval + beta * S1_eval + gamma)*(B_eval + beta * S2_eval + gamma) * (C_eval + beta * S3 + gamma) * ZSH_eval)

R2 = alpha**2 * ((Z - GF(1)) * L1(zeta))

R3 = ZH(zeta) * (TL + zeta**n * TM + zeta**(2*n) * TH)

R = R0 + R1 + R2 - R3

print("\n\n--- R ---")
print(f"R(x) = {R}")

# Compute opening proof polynomial Wz(X)

X_minus_zeta = galois.Poly([1, -zeta], field=GF)
print(X_minus_zeta)

W_zeta = R + \
    v * (A - A_eval) + \
    v**2 * (B - B_eval) + \
    v**3 * (C - C_eval) + \
    v**4 * (S1 - S1_eval) + \
    v**5 * (S2 - S2_eval)

W_zeta = W_zeta // X_minus_zeta

# Compute opening proof polynomial Wzω(X)
X_minus_zeta_omega = galois.Poly([1, -zeta * w], field=GF)

W_omega = (Z - ZSH_eval) // X_minus_zeta_omega

print("\n\n--- W ---")
print(f"Wz(x) = {W_zeta}")
print(f"Wzω(x) = {W_omega}")

# TODO: Compute [Wz] ,[Wzω]

round5 = [W_zeta(tau), W_omega(tau)]
# TODO: compute u from transcript
u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, GF)



--- R ---
R(x) = 117x^15 + 34x^14 + 23x^13 + 126x^12 + 44x^11 + 26x^10 + 122x^9 + 64x^8 + 229x^7 + 234x^6 + 137x^5 + 90x^4 + 46x^3 + 229x^2 + 65x + 207
x + 149


--- W ---
Wz(x) = 117x^14 + 194x^13 + 37x^12 + 156x^11 + 177x^10 + 163x^9 + 212x^8 + 78x^7 + 53x^6 + 69x^5 + 129x^4 + 3x^3 + 228x^2 + 145x + 209
Wzω(x) = 143x^11 + 11x^10 + 212x^9 + 55x^8 + 160x^7 + 59x^6 + 16x^5 + 34x^4 + 23x^3 + 49x^2 + 96x + 227


# Verifier

In [6]:
# Preprocessed
_qm = QM(tau)
_ql = QL(tau)
_qr = QR(tau)
_qo = QO(tau)
_qc = QC(tau)
_s1 = S1(tau)
_s2 = S2(tau)
_s3 = S3(tau)

In [13]:
A_v, B_v, C_v = round1
Z_v = round2[0]
TL_v, TM_v, TH_v = round3
Wz_v, Wz_omega_v = round5

_a, _b, _c, _s1, _s2, _zw = round4

_beta = numbers_to_hash(round1 + [0], GF)
_gamma = numbers_to_hash(round1 + [1], GF)
_alpha = numbers_to_hash(round1 + round2, GF)
_zeta = numbers_to_hash(round1 + round2 + round3, GF)
_v = numbers_to_hash(round1 + round2 + round3 + round4, GF)
_u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, GF)

assert _beta == beta
assert _gamma == gamma
assert _alpha == alpha
assert _zeta == zeta
assert _v == v
assert _u == u

print("\n\n -- Recovered Values --")
print(f"beta = {_beta}")
print(f"gamma = {_gamma}")
print(f"alpha = {_alpha}")
print(f"zeta = {_zeta}")
print(f"v = {_v}")
print(f"u = {_u}")





 -- Recovered Values --
beta = 124
gamma = 122
alpha = 70
zeta = 92
v = 83
u = 80


In [21]:
# Compute zero polynomial evaluation ZH(z) = z^n − 1.
ZH_zeta = zeta**n - GF(1)

# Compute Lagrange polynomial evaluation L1(z) = ω(z^n−1) / n(z−ω)
L1_zeta = ZH_zeta * ((n * (zeta - GF(1)))**-1)

# Compute public input polynomial evaluation PI(z) = Pi∈[l] wiLi(z).
PI = GF(0)

In [25]:
_r0 = PI - L1_zeta * alpha**2 - alpha * (_a + beta * _s1 + gamma) * (_b + beta * _s2 + gamma) * (_c + gamma) * _zw


_D0 = _a * _b * _qm + _a * _ql + _b * _qr + _c * _qo + _qc

_D11 = ((_a + beta * zeta + gamma) * (_b + beta * 2 * zeta + gamma) * (_c + beta * 4 * zeta + gamma) * alpha + L1_zeta * alpha**2 + u) * Z_v

_D12 = (_a + beta * _s1 + gamma) * (_b + beta * _s2 + gamma) * alpha * beta * _zw * _s3

_D2 = ZH_zeta * (TL_v + zeta**n * TM_v + zeta**(2*n) * TH_v)

D = _D0 + _D11 - _D12 - _D2


F = D + v * _a + v**2 * _b + v**3 * _c + v**4 * _s1 + v**5 * _s2

E = -_r0 + v * _a + v**2 * _b + v**3 * _c + v**4 * _s1 + v**5 * _s2 + u * _zw



Wz_v + u * Wz_omega_v == zeta * Wz_v + u * zeta * Wz_omega_v + E * Z_v + F * Z_v**2 + F - E

False